In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - apo_mock_test_dmap
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Test the dust map querying code to try and raise the error where single 
points have multiple lbIndx within the dust map.
'''

__author__ = "James Lane"

In [ ]:
### Imports

import numpy as np
import sys, os, pdb, copy, dill as pickle, time
from astropy import units as apu
from matplotlib import pyplot as plt
from galpy import orbit
from galpy import potential
import mwdust
import healpy
import apomock

### Scale parameters
ro = 8.275
vo = 220
zo = 0.0208 # Bennett+ 2019

### Notebook setup
%matplotlib inline
plt.style.use('../../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

## Plan
Generate `orbit.Orbit` instances with random sets of angles, then use `apomock.APOGEEMock._get_lbIndx()` to try and generate the error

### Functions and preparation

In [ ]:
def sample_angles(n):
    '''sample_angles:
    
    Draw random on-sky angle samples. Even sampling for longitude, and weighted 
    sampling for latitude
    '''
    ll=np.random.random(n)*360.
    bb=np.arccos(2*np.random.random(n)-1)*180./np.pi-90.
    return ll,bb

In [ ]:
# Dust map
dmap = mwdust.Combined19(filter='2 MASS')

In [ ]:
_DEGTORAD = np.pi/180.
def _get_lbIndx(orbs,dmap):
    '''_get_lbIndx:

    Get the index of each sample into the dustmap. The dustmap is 
    arranged as a hierarchical healpix map, with cells at multiple 
    resolutions. Find the index of each sample in this structure

    Args:
        orbs (orbit.Orbit) - phase space samples
        dmap (mwdust.DustMap3D) - Dust map

    Returns:
        lbindx (np.array) - dustmap indices of samples
    '''
    gl = orbs.ll(use_physical=True).value
    gb = orbs.bb(use_physical=True).value
    dist = np.atleast_2d(orbs.dist(use_physical=True).to(apu.kpc).value).T
    # Prepare arrays to hold healpix information for samples
    dmap_nsides = np.array(dmap._nsides)
    pix_arr = np.zeros((len(orbs),len(dmap_nsides)))
    nside_arr = np.repeat(dmap_nsides[:,np.newaxis],len(orbs),axis=1).T
    # Calculate healpix pixels for samples
    for i in range(len(dmap_nsides)):
        pix_arr[:,i] = healpy.pixelfunc.ang2pix(dmap_nsides[i],
                                                (90.-gb)*_DEGTORAD,
                                                gl*_DEGTORAD, nest=True)
    # Calculate healpix u for dust map and samples
    dmap_hpu = (dmap._pix_info['healpix_index'] +\
                4*dmap._pix_info['nside']**2.).astype(int)
    hpu = (pix_arr + 4*nside_arr**2).astype(int)
    # Use searchsorted to match sample u to dust map u
    dmap_hpu_argsort = np.argsort(dmap_hpu)
    dmap_hpu_sorted = dmap_hpu[dmap_hpu_argsort]
    hpu_indx_sorted = np.searchsorted(dmap_hpu_sorted,hpu)
    hpu_indx = np.take(dmap_hpu_argsort, hpu_indx_sorted, mode="clip")
    hpu_mask = dmap_hpu[hpu_indx] != hpu
    hpu_ma = np.ma.array(hpu_indx, mask=hpu_mask)
    return hpu_ma

### Get lbIndx values

In [ ]:
n_samples = int(1e5)
ll,bb = sample_angles(n_samples)
dist = np.ones_like(ll)
mul,mub,vlos = np.zeros_like(ll), np.zeros_like(ll), np.zeros_like(ll)
vxvvs = np.array([ll,bb,dist,mul,mub,vlos]).T
orbs = orbit.Orbit(vxvvs,lb=True,ro=ro,vo=vo)
# Quick sanity checks
assert np.all(np.fabs(ll-orbs.ll().value)<1e-8), 'll not correct'
assert np.all(np.fabs(bb-orbs.bb().value)<1e-8), 'bb not correct'

In [ ]:
# Get lbIndx
hpu_ma = _get_lbIndx(orbs,dmap)
lbIndx = hpu_ma.data[~hpu_ma.mask]
print(str(len(lbIndx))+' lbIndx for '+str(len(orbs))+' orbits')

In [ ]:
# Identify the orbits where multiple matches occured
multiple_lbIndx_mask = np.sum(~hpu_ma.mask,axis=1)>1
where_multiple_lbIndx = np.where(multiple_lbIndx_mask)[0]
hpu_ma[where_multiple_lbIndx]

In [ ]:
x = np.array([1,2,3,4,5,6,7])
x[x>5]*=2

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ll_plot = ll[where_multiple_lbIndx]
ll_plot[ll_plot>180.]-=360.
bb_plot = bb[where_multiple_lbIndx]
ax.scatter(ll_plot, bb_plot, s=8., color='Black')
ax.set_xlabel(r'$\ell$')
ax.set_ylabel(r'$b$')
fig.show()

### Visualize the Healpix maps

In [ ]:
dir(dmap)

In [ ]:
dmap._pix_info

In [ ]:
dmap._pix_info

In [ ]:
dmap._nsides

In [ ]:
nsides = dmap._nsides
# fig = plt.figure(figsize=(5,20))
# axs = fig.subplots(nrows=5, ncols=1)

for i in range(len(nsides)):
    hpi = dmap._pix_info['healpix_index'][dmap._pix_info['nside']==nsides[i]]
    m = np.zeros(healpy.nside2npix(nsides[i]))
    m[hpi]=1.
    healpy.mollview(m,nest=True)
    fig = plt.gcf()
    fig.suptitle('Nsides = '+str(nsides[i]),x=0.7)
###i

## Process a single map evaluated at resolution Nside=1024
So the pixels at two different resolutions separated by a single order, i.e. 1024 to 512, have a difference in the range spanned by their indices of a factor of 4. So given an index $p_{1}$ at resolution $k_{1}$, the indices of the descendant pixels of $p$ at resolution $k_{2}$ are:

$p_{2} = 4^{k_{2}-k_{1}}p_{1} + [0,...,4^{k_{2}-k_{1}}]$

In [ ]:
def get_healpix_descendents(p1,k1,k2):
    '''Take a healpix array p1 at order k1 and get all descendents at a higher 
    order k2
    '''
    # Make the new array
    dk = int(k2-k1)
    dres = 4**dk
    p2 = np.repeat(p1*dres,dres).astype(int)
    pdesc = np.tile(np.arange(0,dres,1),p1.shape[0]).astype(int)
    # pdb.set_trace()
    p2 += pdesc
    return p2

In [ ]:
nside_ks = np.log2(nsides).astype(int)
ps = []
for i in range(len(nsides)):
    p1 = dmap._pix_info['healpix_index'][dmap._pix_info['nside']==nsides[i]]
    if i == 0: 
        ps.append(p1)
        continue  # Already at resolution 1024
    p2 = get_healpix_descendents(p1,nside_ks[i],np.log2(1024).astype(int))
    ps.append(p2)

In [ ]:
p10 = np.zeros(healpy.nside2npix(1024),dtype=int)
for i in range(len(nsides)):
    p10[ps[i]] += 1

In [ ]:
healpy.mollview(p10,nest=True,min=0,max=2)